# Solvers for [Wordle](https://www.powerlanguage.co.uk/wordle/) and [Absurdle](https://qntm.org/files/absurdle/absurdle.html)
## Author: Keshav Gupta (keshav21@mit.edu)
### First, lets load up our wordlists.

In [ ]:
with open("wordle-allowed-guesses.txt") as f:
    allowedGuesses = [line.split("\n")[0] for line in f]
with open("wordle-answers-alphabetical.txt") as f:
    allowedAnswers = [line.split("\n")[0] for line in f]
allGuesses = allowedGuesses[:] + allowedAnswers[:]

### Next, lets define some helper functions to play a round, etc.

In [ ]:
def getOutcome(guess, secret):
    outcome = ['x' for _ in range(5)]
    secretD = {}
    for letter in secret:
        if letter in secretD:
            secretD[letter] += 1
        else:
            secretD[letter] = 1
    for i in range(5):
        if guess[i] == secret[i]:
            secretD[secret[i]] -= 1
            outcome[i] = 'g'
    for i in range(5):
        if outcome[i] != 'g' and guess[i] in secretD and secretD[guess[i]] > 0:
            outcome[i] = 'y'
            secretD[guess[i]] -= 1
    return "".join(outcome)

In [ ]:
def genAllOutcomes(i):
    if i <= 0:
        yield ''
    else:
        for outcome in genAllOutcomes(i-1):
            yield 'x' + outcome
            yield 'g' + outcome
            yield 'y' + outcome

allOutcomes = list(genAllOutcomes(5))

In [ ]:
def filterToBuckets(guess, dictionary):
    outcomes = {o: [] for o in allOutcomes}
    for word in dictionary:
        outcome = getOutcome(guess, word)
        outcomes[outcome].append(word)
    return outcomes

### These functions pretty-print everything.

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def fancyPrint(outcome, guess, end = "\n"):
    print("Outcome: ", end = "")
    for i in range(5):
        if outcome[i] == 'x':
            print(guess[i], end = "")
        elif outcome[i] == 'g':
            print(bcolors.OKGREEN + guess[i] + bcolors.ENDC, end = "")
        elif outcome[i] == 'y':
            print(bcolors.WARNING + guess[i] + bcolors.ENDC, end = "")
    print("", end=end)

def fancyBucketPrint(buckets):
    outcomesList = list(buckets.keys())
    outcomesList.sort(reverse = True, key = lambda t: len(buckets[t]))
    print("{", end = "")
    for outcome in outcomesList[:5]:
        print(f"{outcome}: {len(buckets[outcome])}, ", end = "")
    if len(outcomesList) > 5:
        print("... }")
    else:
        print("}")


# Wordle
### Let's simulate a round of Wordle.

In [ ]:
import random

def interactWordle():
    secretWord = random.choice(allowedAnswers)
    allLetters = list("etaoinshrdlcumwfgypbvkjxqz") # by frequency
    while True:
        guess = input("Enter your guess:")
        if guess == "":
            break
        else:
            if guess in allGuesses:
                outcome = getOutcome(guess, secretWord)
                fancyPrint(outcome, guess)
                if outcome == "ggggg":
                    print("You win!")
                else:
                    for letter in guess:
                        try:
                            i = allLetters.index(letter)
                            allLetters.pop(i)
                        except ValueError:
                            pass
                    print("Unused letters:", "".join(allLetters))
                    print("***************")
            else:
                print("Guess not in dictionary.")

# interactWordle()

### Let's try minmax to find the best Wordle guesses.

In [ ]:
def iterateWordle(dictionary):
    minMaxBucket = len(dictionary)
    if minMaxBucket == 1:
        return dictionary[0], {"ggggg": dictionary[0]}
    else:
        bestGuess = ""
        bestBuckets = dict()
        for guess in allGuesses:
            buckets = filterToBuckets(guess, dictionary)
            maxBucketSize = max([len(buckets[t]) for t in buckets])
            if maxBucketSize < minMaxBucket:
                minMaxBucket = maxBucketSize
                bestGuess = guess
                bestBuckets = buckets
        return bestGuess, bestBuckets

### Memoize the first two guesses to make the solver faster.

In [ ]:
def memoize():
    firstGuess = "aesir"
    buckets = filterToBuckets(firstGuess, allowedAnswers)
    secondGuesses = dict()
    for outcome in buckets:
        print(f"Working on outcome {outcome}.")
        bg, _ = iterateWordle(buckets[outcome])
        secondGuesses[outcome] = bg
    print(secondGuesses)

# memoize()

In [ ]:
firstGuess = 'aesir'
secondGuesses = {'xxxxx': 'bludy', 'gxxxx': 'glout', 'yxxxx': 'canty', 'xgxxx': 'culty', 'ggxxx': '', 'ygxxx': 'typal', 'xyxxx': 'nould', 'gyxxx': 'blate', 'yyxxx': 'cable', 'xxgxx': 'altho', 'gxgxx': 'assay', 'yxgxx': 'hotty', 'xggxx': 'azote', 'gggxx': '', 'yggxx': '', 'xygxx': 'acton', 'gygxx': 'asset', 'yygxx': 'bitch', 'xxyxx': 'cloot', 'gxyxx': 'abacs', 'yxyxx': 'latch', 'xgyxx': 'antae', 'ggyxx': '', 'ygyxx': 'flyte', 'xyyxx': 'stoln', 'gyyxx': 'abash', 'yyyxx': 'shakt', 'xxxgx': 'clomp', 'gxxgx': 'anted', 'yxxgx': 'abamp', 'xgxgx': 'aband', 'ggxgx': '', 'ygxgx': 'media', 'xyxgx': 'bonce', 'gyxgx': '', 'yyxgx': 'email', 'xxggx': 'abamp', 'gxggx': '', 'yxggx': 'acnes', 'xgggx': '', 'ggggx': '', 'ygggx': '', 'xyggx': '', 'gyggx': '', 'yyggx': '', 'xxygx': 'ablet', 'gxygx': '', 'yxygx': 'actin', 'xgygx': '', 'ggygx': '', 'ygygx': 'sepia', 'xyygx': 'aahed', 'gyygx': '', 'yyygx': '', 'xxxyx': 'clint', 'gxxyx': 'nopal', 'yxxyx': 'talon', 'xgxyx': 'adhan', 'ggxyx': '', 'ygxyx': '', 'xyxyx': 'cline', 'gyxyx': 'blive', 'yyxyx': 'amene', 'xxgyx': 'schmo', 'gxgyx': '', 'yxgyx': 'vista', 'xggyx': '', 'gggyx': '', 'yggyx': '', 'xygyx': 'issue', 'gygyx': 'aisle', 'yygyx': '', 'xxyyx': 'skint', 'gxyyx': 'amiss', 'yxyyx': 'agast', 'xgyyx': 'aahed', 'ggyyx': '', 'ygyyx': '', 'xyyyx': 'pling', 'gyyyx': 'aside', 'yyyyx': '', 'xxxxg': 'furth', 'gxxxg': 'adbot', 'yxxxg': 'novum', 'xgxxg': 'flout', 'ggxxg': '', 'ygxxg': 'aband', 'xyxxg': 'compt', 'gyxxg': 'abaft', 'yyxxg': 'grypt', 'xxgxg': '', 'gxgxg': '', 'yxgxg': '', 'xggxg': '', 'gggxg': '', 'yggxg': '', 'xygxg': 'aalii', 'gygxg': '', 'yygxg': '', 'xxyxg': 'scour', 'gxyxg': '', 'yxyxg': 'acton', 'xgyxg': 'ablow', 'ggyxg': '', 'ygyxg': '', 'xyyxg': 'autos', 'gyyxg': '', 'yyyxg': 'aheap', 'xxxgg': 'choir', 'gxxgg': '', 'yxxgg': 'aahed', 'xgxgg': '', 'ggxgg': '', 'ygxgg': '', 'xyxgg': 'their', 'gyxgg': '', 'yyxgg': '', 'xxggg': '', 'gxggg': '', 'yxggg': '', 'xgggg': '', 'ggggg': '', 'ygggg': '', 'xyggg': '', 'gyggg': '', 'yyggg': '', 'xxygg': '', 'gxygg': '', 'yxygg': 'stair', 'xgygg': '', 'ggygg': '', 'ygygg': '', 'xyygg': '', 'gyygg': '', 'yyygg': '', 'xxxyg': 'avion', 'gxxyg': '', 'yxxyg': 'abcee', 'xgxyg': '', 'ggxyg': '', 'ygxyg': '', 'xyxyg': 'divan', 'gyxyg': 'aider', 'yyxyg': '', 'xxgyg': 'visor', 'gxgyg': '', 'yxgyg': '', 'xggyg': '', 'gggyg': '', 'yggyg': '', 'xygyg': 'amowt', 'gygyg': '', 'yygyg': '', 'xxyyg': '', 'gxyyg': '', 'yxyyg': '', 'xgyyg': '', 'ggyyg': '', 'ygyyg': '', 'xyyyg': 'skier', 'gyyyg': '', 'yyyyg': '', 'xxxxy': 'conto', 'gxxxy': 'anomy', 'yxxxy': 'canto', 'xgxxy': 'corby', 'ggxxy': '', 'ygxxy': 'ryals', 'xyxxy': 'dropt', 'gyxxy': 'loure', 'yyxxy': 'cadge', 'xxgxy': 'rusty', 'gxgxy': 'arson', 'yxgxy': 'raspy', 'xggxy': 'reset', 'gggxy': '', 'yggxy': '', 'xygxy': '', 'gygxy': '', 'yygxy': '', 'xxyxy': 'curst', 'gxyxy': 'artsy', 'yxyxy': 'scath', 'xgyxy': 'kerve', 'ggyxy': '', 'ygyxy': '', 'xyyxy': 'scowp', 'gyyxy': 'arose', 'yyyxy': 'canst', 'xxxgy': 'boult', 'gxxgy': 'acrid', 'yxxgy': 'balti', 'xgxgy': 'compt', 'ggxgy': '', 'ygxgy': '', 'xyxgy': '', 'gyxgy': '', 'yyxgy': '', 'xxggy': '', 'gxggy': '', 'yxggy': '', 'xgggy': 'resin', 'ggggy': '', 'ygggy': '', 'xyggy': '', 'gyggy': '', 'yyggy': '', 'xxygy': 'aargh', 'gxygy': '', 'yxygy': '', 'xgygy': 'serif', 'ggygy': '', 'ygygy': '', 'xyygy': '', 'gyygy': '', 'yyygy': '', 'xxxyy': 'linky', 'gxxyy': '', 'yxxyy': 'caird', 'xgxyy': 'aahed', 'ggxyy': '', 'ygxyy': '', 'xyxyy': 'timed', 'gyxyy': 'afire', 'yyxyy': 'irate', 'xxgyy': 'risky', 'gxgyy': '', 'yxgyy': '', 'xggyy': '', 'gggyy': '', 'yggyy': '', 'xygyy': 'risen', 'gygyy': '', 'yygyy': '', 'xxyyy': 'chirk', 'gxyyy': '', 'yxyyy': '', 'xgyyy': '', 'ggyyy': '', 'ygyyy': '', 'xyyyy': 'adhan', 'gyyyy': 'arise', 'yyyyy': 'raise'}

In [ ]:
def solveWordle(dictionary = None):
    if dictionary is None:
        dictionary = allowedAnswers
        attempt = 0
    else:
        attempt = 2
    while True:
        if attempt == 0:
            bestGuess = "aesir"
            bestBuckets = filterToBuckets(bestGuess, dictionary)
        elif attempt == 1:
            bestGuess = secondGuesses[outcome]
            bestBuckets = filterToBuckets(bestGuess, dictionary)
        elif attempt >= 2:
            bestGuess, bestBuckets = iterateWordle(dictionary)
        outcome = input(f"Outcome when you try \"{bestGuess}\" (combination of x, y, and g):")
        attempt += 1
        if not outcome or outcome == "ggggg":
            break
        fancyPrint(outcome, bestGuess)
        dictionary = bestBuckets[outcome]

# solveWordle()

### A solver for Wordle that picks up mid game.

In [ ]:
def solveWordleContinuation(guessOutcomePairs):
    dictionary = allowedAnswers
    for (g, o) in guessOutcomePairs:
        dictionary = filterToBuckets(g, dictionary)[o]
    solveWordle(dictionary)

# solveWordleContinuation([('raise', 'yxxxx'), ('clout', 'xxyyx'), ('rough', 'yyyxy')])

### Now, lets evaluate this solver by the number of turns it takes for each possible answer.

In [ ]:
def evaluateSolver():
    attemptCounts = []
    for secret in allowedAnswers:
        attempt = 0
        dictionary = allowedAnswers
        while True:
            if attempt == 0:
                bestGuess = "aesir"
                bestBuckets = filterToBuckets(bestGuess, dictionary)
            elif attempt == 1:
                bestGuess = secondGuesses[outcome]
                bestBuckets = filterToBuckets(bestGuess, dictionary)
            elif attempt >= 2:
                bestGuess, bestBuckets = iterateWordle(dictionary)
            outcome = getOutcome(bestGuess, secret)
            attempt += 1
            if outcome == 'ggggg':
                attemptCounts.append(attempt)
                break
            dictionary = bestBuckets[outcome]
    print(attemptCounts)

# evaluateSolver()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plotAttemptCounts():
    # attempt counts obtained from before
    attemptCounts = [3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 4, 3, 3, 4, 2, 3, 3, 4, 3, 4, 3, 4, 4, 4, 3, 4, 3, 4, 2, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4, 4, 3, 4, 2, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 4, 2, 3, 4, 4, 4, 3, 3, 3, 4, 3, 3, 3, 4, 4, 3, 4, 4, 3, 3, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 4, 2, 2, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 3, 4, 5, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 4, 4, 5, 4, 5, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 5, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 5, 5, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 3, 4, 5, 5, 4, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 5, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 3, 4, 4, 4, 3, 5, 5, 4, 4, 4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3, 5, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 3, 4, 4, 4, 5, 4, 4, 4, 3, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 5, 4, 3, 3, 5, 3, 5, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 3, 4, 4, 5, 5, 3, 4, 4, 4, 4, 3, 3, 4, 5, 5, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 5, 4, 3, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 3, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 4, 4, 3, 4, 4, 4, 3, 2, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 3, 4, 3, 4, 4, 4, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 4, 4, 3, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 5, 5, 5, 5, 4, 5, 5, 3, 3, 4, 4, 4, 3, 5, 5, 4, 4, 4, 3, 5, 4, 5, 5, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 3, 4, 4, 5, 3, 4, 4, 5, 5, 5, 4, 4, 4, 4, 3, 4, 5, 4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 5, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4, 3, 4, 4, 4, 4, 5, 5, 3, 5, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 5, 4, 3, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 4, 5, 4, 4, 4, 4, 5, 4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 5, 5, 3, 4, 4, 4, 4, 4, 5, 5, 5, 4, 5, 5, 4, 3, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 3, 5, 4, 4, 4, 4, 4, 3, 4, 5, 3, 5, 3, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 5, 4, 3, 4, 4, 5, 5, 5, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 4, 4, 3, 3, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 2, 4, 3, 2, 4, 4, 5, 5, 4, 4, 4, 4, 5, 4, 5, 5, 5, 3, 4, 4, 4, 4, 4, 3, 4, 3, 4, 5, 4, 4, 4, 4, 3, 5, 3, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 3, 3, 3, 4, 4, 3, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 3, 4, 5, 3, 4, 5, 4, 4, 3, 3, 3, 5, 5, 5, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 4, 3, 4, 4, 3, 4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 3, 4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 2, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 5, 3, 4, 4, 4, 3, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 5, 4, 4, 5, 4, 3, 4, 4, 4, 3, 3, 5, 4, 4, 5, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 4, 4, 3, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 5, 3, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 3, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 3, 4, 5, 4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 3, 4, 3, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 3, 4, 5, 3, 4, 4, 5, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 3, 3, 3, 5, 4, 3, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 5, 4, 3, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 5, 3, 3, 5, 3, 3, 4, 3, 2, 4, 5, 5, 4, 3, 3, 4, 4, 5, 2, 3, 4, 4, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 4, 3, 4, 4, 4, 2, 2, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 5, 2, 3, 2, 4, 4, 3, 3, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 3, 4, 4, 2, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 3, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 2, 3, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 4, 3, 4, 4, 2, 2, 3, 3, 4, 4, 3, 3, 3, 4, 4, 3, 3, 4, 4, 4, 3, 4, 4, 5, 4, 4, 4, 4, 5, 4, 3, 3, 4, 4, 4, 3, 5, 5, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 4, 4, 3, 2, 4, 4, 4, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 3, 4, 5, 5, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 3, 4, 3, 4, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 3, 5, 4, 3, 3, 2, 3, 4, 4, 4, 5, 5, 5, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 3, 3, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 3, 4, 4, 5, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 3, 4, 5, 4, 5, 5, 3, 3, 3, 3, 3, 3, 4, 3, 5, 5, 3, 3, 4, 3, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 3, 4, 4, 2, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 3, 5, 5, 4, 5, 4, 3, 3, 4, 4, 5, 3, 4, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 5, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 3, 4, 4, 3, 4, 3, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 3, 3, 4, 3, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 2, 2, 3, 4, 4, 5, 4, 3, 4, 4, 4, 4, 5, 4, 4, 3, 5, 4, 4, 3, 3, 3, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 5, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 3, 3, 4, 5, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4, 3, 3, 4, 4, 5, 5, 3, 3, 4, 4, 3, 4, 4, 4, 5, 4, 4, 5, 3, 4, 3, 4, 4, 4, 4, 3, 4]
    plt.hist(attemptCounts, bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5])
    plt.title(f"Attempt Count Stats - Average: {np.mean(attemptCounts):0.2f}, Std. Dev: {np.std(attemptCounts):0.2f}, Max: {max(attemptCounts)}")
    plt.xlabel("Number of attempts")
    plt.ylabel("Frequency of words")
    plt.savefig("wordle_solver_stats.pdf")

# plotAttemptCounts()

# Absurdle: the adversarial variant of Wordle
### First some helper functions to generate outcomes and filter a word list by guess.

In [ ]:
def outcomeIsWorse(outcome, worstOutcome):
    if outcome.count('g') < worstOutcome.count('g'):
        return True
    elif outcome.count('g') == worstOutcome.count('g'):
        return outcome.count('y') < worstOutcome.count('y')
    else:
        return False

In [ ]:
def filterOnGuess(guess, currentWords):
    """
    guess: one of the allowed guesses
    currentWords: list of words that are still the possible secrets

    output: returned as (outcome, remainingWords)
    outcome: 5 character long string of ('x', 'g', 'y')
    remainingWords: list of remaining words
    """
    outcomes = {o: [] for o in allOutcomes}
    for word in currentWords:
        outcome = getOutcome(guess, word)
        outcomes[outcome].append(word)
    worstOutcome = ''
    mostSecrets = 0
    for outcome in outcomes:
        thisSecrets = len(outcomes[outcome])
        if thisSecrets > mostSecrets or thisSecrets == mostSecrets and outcomeIsWorse(outcome, worstOutcome):
            mostSecrets = thisSecrets
            worstOutcome = outcome
    return worstOutcome, outcomes[worstOutcome], outcomes

### Simulate a round of Absurdle.

In [ ]:
def interactAbsurdle():
    currentWords = allGuesses[:]
    allLetters = list("etaoinshrdlcumwfgypbvkjxqz") # by frequency
    while True:
        print("***************")
        guess = input("Enter your guess:")
        if guess == "":
            break
        else:
            if guess in allGuesses:
                outcome, currentWords, _ = filterOnGuess(guess, currentWords)
                fancyPrint(outcome, guess)
                if outcome == "ggggg":
                    print("You win!")
                else:
                    for letter in guess:
                        try:
                            i = allLetters.index(letter)
                            allLetters.pop(i)
                        except ValueError:
                            pass
                    print("Unused letters:", "".join(allLetters))
                    print("Number of words remaining:", len(currentWords))
                    print("***************")
            else:
                print("Guess not in dictionary.")

# interactAbsurdle()

### Now, let's try to greedily find the best guesses. Depending on your compute, these functions might all take a couple of minutes to run.

In [ ]:
def iterateAbsurdle(dictionary):
    lastMin = len(dictionary)
    bestGuess = None
    bestOutcome = 'ggggg'
    bestCurrentWords = []
    for guess in allGuesses:
        outcome, currentWords, _ = filterOnGuess(guess, dictionary)
        if len(currentWords) <= lastMin:
            bestGuess = guess
            bestCurrentWords = currentWords
            bestOutcome = outcome
            lastMin = len(currentWords)
    fancyPrint(bestOutcome, bestGuess)
    if len(bestCurrentWords) == 1:
        print("***************************************************************")
        print("Done. ", end = "")
        fancyPrint('ggggg', currentWords[0])
        return []
    else:
        print("***************************************************************")
        return bestCurrentWords

In [ ]:
def solveAbsurdleGreedy():
    currentWords = allowedAnswers[:]
    while currentWords:
        currentWords = iterateAbsurdle(currentWords)

# solveAbsurdleGreedy()

### Therefore the best greedily maximized Absurdle guess sequence is: `raise`, `duply`, `witch`, `zonal`, `uncut`. Note that there exists a four guess solution to Absurdle, but that would require a tree search.

### Solve Absurdle mid-game (after the user has already made a few guesses):

In [ ]:
def solveAbsurdleContinuation(alreadyGuessed):
    currentWords = allowedAnswers[:]
    for guess in alreadyGuessed:
        outcome, currentWords, _ = filterOnGuess(guess, currentWords)
        fancyPrint(outcome, guess)
    print("***************************************************************")
    while currentWords:
        currentWords = iterateAbsurdle(currentWords)

# solveAbsurdleContinuation(["raise", "witch", "duply"])

### Now, let's try to solve the game in *Challenge* mode (where the final guess has to be a specific word)

In [ ]:
def iterateAbsurdleChallenge(target, dictionary, prevGuesses):
    lastMin = len(dictionary)
    bestGuess = None
    bestOutcome = 'ggggg'
    bestCurrentWords = []
    for guess in allGuesses:
        if guess not in prevGuesses:
            outcome, currentWords, buckets = filterOnGuess(guess, dictionary)
            if target in currentWords:
                # the largest bucket must contain the target word and must be one larger than the rest
                if len(currentWords) < lastMin:
                    lens = [len(buckets[t]) for t in buckets]
                    lens.sort(reverse=True)
                    if lens[0] > lens[1]:
                        bestGuess = guess
                        bestOutcome = outcome
                        bestCurrentWords = currentWords
                        lastMin = len(bestCurrentWords)
    fancyPrint(bestOutcome, bestGuess)
    if len(bestCurrentWords) == 1:
        print("***************************************************************")
        print("Done. ", end = "")
        fancyPrint('ggggg', bestCurrentWords[0])
        return bestCurrentWords[0], bestCurrentWords
    else:
        print("***************************************************************")
        return bestGuess, bestCurrentWords

In [ ]:
def solveAbsurdleChallenge(targetSecret):
    currentWords = allowedAnswers[:]
    prevGuesses = []
    while len(currentWords) > 2:
        bg, currentWords = iterateAbsurdleChallenge(targetSecret, currentWords, prevGuesses)
        prevGuesses.append(bg)
    for guess in allGuesses:
        if guess not in prevGuesses:
            outcome, finalWords, buckets = filterOnGuess(guess, currentWords)
            if len(finalWords) == 1 and finalWords[0] == targetSecret:
                prevGuesses.append(guess)
                fancyPrint(outcome, guess)
                print("***************************************************************")
                break
    fancyPrint("ggggg", targetSecret)


# solveAbsurdleChallenge("rocky")